In [ ]:
import os
from openai import OpenAI
import pandas as pd
import base64
import re
import time
import json

# 初始化OpenAI客户端
client = OpenAI(
    api_key="",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

imgs_questions = pd.read_csv("../Dataset/SoMDataset/Relational/3_color_size_shape/3_color_size_shape.csv")

# 获取所有描述文本（假设文本在第二列）
text_list = imgs_questions.iloc[17:, 1]  # 获取文本列

# 本地图片文件夹路径
folder_path = "../Dataset/SoMDataset/Relational/3_color_size_shape/imgs" 
# 将图片转换为base64编码
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
# 获取文件夹中的所有图片
image_files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))],
    key=lambda x: int(re.match(r'(\d+)', x).group(1))
)

# 遍历图片URL和文本
for idx, filename in enumerate(image_files):
    image_path = os.path.join(folder_path, filename)
    text = text_list.iloc[idx] # 假设imgurl是本地文件路径
    # 将图片转换为base64编码
    base64_image = encode_image(image_path)
    # 构建消息
    messages = [{
        "role": "user",
        "content": [
            {"type": "text", "text": text+",找出正确物体的序号。"},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                }
            }
        ]
    }]

    # 调用API
    completion = client.chat.completions.create(
        model="qwen-vl-max",
        messages=messages
    )

    # 打印API返回的结果
    print(json.loads(completion.model_dump_json())['choices'][0]['message']['content'])
    time.sleep(2)   
    
